In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from scipy.stats import zscore
from wordcloud import WordCloud
import nltk

In [2]:
df = pd.read_excel("C:\\Users\\tanis\\OneDrive\\Desktop\\cleaned_data_with_tags.xlsx")
df.head()


,VIN,TRANSACTION_ID,CORRECTION_VERBATIM,CUSTOMER_VERBATIM,REPAIR_DATE,CAUSAL_PART_NM,GLOBAL_LABOR_CODE_DESCRIPTION,PLATFORM,BODY_STYLE,VPPC,...,SRC_VER_NBR,TRANSACTION_CNTR,MEDIA_FLAG,VIN_MODL_DESGTR,LINE_SERIES,LAST_KNOWN_DELVRY_TYPE_CD,NON_CAUSAL_PART_QTY,SALES_REGION_CODE,processed_text,tags
0,3hcfdde89sh220903,13021,replaced steering wheel now okay,steering wheel coming apart,2024-01-02,wheel asm-strg *jet black,steering wheel replacement,full-size trucks,crew cab,t1ccf,...,6,1,n,cf10543,NaN,21.0,0,1,steering wheel coming apart,apart coming coming apart
1,3hcfdfel3sh241701,13021,steering wheel replacement,customer states the lettering and finish on th...,2024-01-04,wheel asm-strg *jet black,steering wheel replacement,full-size trucks,crew cab,t1ccf,...,6,1,y,cf10543,NaN,10.0,0,1,customer states lettering finish steering whee...,coming coming plant customer
2,1hrffhel1rz181474,13021,replaced steering message no longer displayed,c/s: customer states the service driver assist...,2024-01-05,wheel asm-strg *jet black,steering wheel replacement,full-size trucks,crew cab,t1cgf,...,8,1,n,tf10543,NaN,10.0,0,1,customer states service driver assist system m...,assist assist message customer
3,3hrffhed7rh167541,13026,remove and replace steering wheel wire harness,horn and steering wheel switches are inoperable,2024-01-05,harness asm-strg whl horn sw wrg,steering wheel horn switch wiring harness repl...,full-size trucks,crew cab,t1cgf,...,6,1,n,tf10543,NaN,16.0,0,1,horn steering wheel switches inoperable,horn horn steering inoperable
4,3hcfdfed4sh352945,13021,replaced steering wheel,special order part [ steering wheel heated ste...,2024-01-05,wheel asm-strg *jet black,steering wheel replacement,full-size trucks,crew cab,t1ccf,...,4,1,n,cf10543,NaN,15.0,0,1,special order part steering wheel heated steer...,steering steering wheel wheel


In [3]:

print("Duplicates:", df.duplicated().sum())


Duplicates: 0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   VIN                            75 non-null     object        
 1   TRANSACTION_ID                 75 non-null     int64         
 2   CORRECTION_VERBATIM            75 non-null     object        
 3   CUSTOMER_VERBATIM              75 non-null     object        
 4   REPAIR_DATE                    75 non-null     datetime64[ns]
 5   CAUSAL_PART_NM                 75 non-null     object        
 6   GLOBAL_LABOR_CODE_DESCRIPTION  75 non-null     object        
 7   PLATFORM                       75 non-null     object        
 8   BODY_STYLE                     75 non-null     object        
 9   VPPC                           75 non-null     object        
 10  PLANT                          75 non-null     object        
 11  BUILD_COUNTRY        

In [5]:
df.describe()

,TRANSACTION_ID,REPAIR_DATE,DEALER_REGION,REPAIR_AGE,KM,COMPLAINT_CD_CSI,ORD_SELLING_SRC_CD,GLOBAL_LABOR_CODE,CAMPAIGN_NBR,REPORTING_COST,TOTALCOST,LBRCOST,TRANSMISSION_SOURCE_PLANT,SRC_TXN_ID,SRC_VER_NBR,TRANSACTION_CNTR,LAST_KNOWN_DELVRY_TYPE_CD,NON_CAUSAL_PART_QTY,SALES_REGION_CODE
count,75.000000,75,75.0,75.000000,75.000000,75.0,75.000000,75.000000,0.0,75.000000,75.000000,75.0000,7.500000e+01,7.500000e+01,75.000000,75.0,75.000000,75.0,75.0
mean,13035.240000,2024-01-23 13:45:36,1.0,14.520000,23625.453333,0.0,22.880000,117.066667,NaN,499.678533,515.089104,87.6480,1.687060e+08,2.816087e+09,5.466667,1.0,13.015102,0.0,1.0
min,13021.000000,2024-01-02 00:00:00,1.0,0.000000,3.000000,0.0,11.000000,20.000000,NaN,27.690000,27.690000,22.9300,2.878270e+05,2.808882e+09,2.000000,1.0,10.000000,0.0,1.0
25%,13029.000000,2024-01-15 00:00:00,1.0,5.000000,8775.000000,0.0,13.000000,130.000000,NaN,315.930000,364.025000,61.9850,2.878270e+05,2.809429e+09,4.000000,1.0,10.000000,0.0,1.0
50%,13036.000000,2024-01-26 00:00:00,1.0,12.000000,17774.000000,0.0,13.000000,130.000000,NaN,432.680000,457.980000,77.6900,8.042172e+06,2.820191e+09,4.000000,1.0,10.000000,0.0,1.0
75%,13040.000000,2024-02-02 00:00:00,1.0,21.000000,34730.500000,0.0,48.000000,130.000000,NaN,520.495000,561.162128,100.1350,1.676383e+08,2.820841e+09,6.000000,1.0,16.000000,0.0,1.0
max,13065.000000,2024-02-07 00:00:00,1.0,49.000000,80805.000000,0.0,48.000000,130.000000,NaN,1712.850000,1480.020000,244.5400,8.282984e+08,2.823000e+09,16.000000,1.0,32.000000,0.0,1.0
std,8.740771,NaN,0.0,12.134784,19113.031336,0.0,15.825877,31.826104,NaN,347.994890,326.268667,40.9813,3.069225e+08,5.674349e+06,2.637225,0.0,4.534054,0.0,0.0


In [7]:
df.columns

Index(['VIN', 'TRANSACTION_ID', 'CORRECTION_VERBATIM', 'CUSTOMER_VERBATIM',
       'REPAIR_DATE', 'CAUSAL_PART_NM', 'GLOBAL_LABOR_CODE_DESCRIPTION',
       'PLATFORM', 'BODY_STYLE', 'VPPC', 'PLANT', 'BUILD_COUNTRY',
       'LAST_KNOWN_DLR_NAME', 'LAST_KNOWN_DLR_CITY', 'REPAIRING_DEALER_CODE',
       'DEALER_NAME', 'REPAIR_DLR_CITY', 'STATE', 'DEALER_REGION',
       'REPAIR_DLR_POSTAL_CD', 'REPAIR_AGE', 'KM', 'COMPLAINT_CD_CSI',
       'COMPLAINT_CD', 'VEH_TEST_GRP', 'COUNTRY_SALE_ISO',
       'ORD_SELLING_SRC_CD', 'OPTN_FAMLY_CERTIFICATION',
       'OPTF_FAMLY_EMISSIOF_SYSTEM', 'GLOBAL_LABOR_CODE',
       'TRANSACTION_CATEGORY', 'CAMPAIGN_NBR', 'REPORTING_COST', 'TOTALCOST',
       'LBRCOST', 'ENGINE', 'ENGINE_DESC', 'TRANSMISSION', 'TRANSMISSION_DESC',
       'ENGINE_SOURCE_PLANT', 'ENGINE_TRACE_NBR', 'TRANSMISSION_SOURCE_PLANT',
       'TRANSMISSION_TRACE_NBR', 'SRC_TXN_ID', 'SRC_VER_NBR',
       'TRANSACTION_CNTR', 'MEDIA_FLAG', 'VIN_MODL_DESGTR', 'LINE_SERIES',
       'LAST_KNOWN_D

In [10]:
columns_to_drop = [
    'CAMPAIGN_NBR', 
    'ENGINE_SOURCE_PLANT', 
    'LINE_SERIES', 
    'REPAIR_DLR_POSTAL_CD', 
    'ENGINE_TRACE_NBR', 
    'TRANSMISSION_SOURCE_PLANT'
]
df.drop(columns=columns_to_drop, axis=1, inplace=True)


df['REPAIR_DATE'] = pd.to_datetime(df['REPAIR_DATE'], errors='coerce')  
df.fillna({'REPORTING_COST': 0, 'TOTALCOST': 0}, inplace=True) 
df.dropna(subset=['VIN', 'TRANSACTION_ID'], inplace=True)  

df.drop_duplicates(inplace=True)


df['KM'] = pd.to_numeric(df['KM'], errors='coerce')  # Convert to numeric
df['TOTALCOST'] = pd.to_numeric(df['TOTALCOST'], errors='coerce')


df = df[(df['KM'] >= 0) & (df['KM'] <= 500000)]  
df = df[(df['TOTALCOST'] >= 0) & (df['TOTALCOST'] <= 100000)]  

# Save the cleaned data back to a new file
df.to_csv('cleaned_data.csv', index=False)

# Display the cleaned data
print(df.head())

                 VIN  TRANSACTION_ID  \
0  3hcfdde89sh220903           13021   
1  3hcfdfel3sh241701           13021   
2  1hrffhel1rz181474           13021   
3  3hrffhed7rh167541           13026   
4  3hcfdfed4sh352945           13021   

                              CORRECTION_VERBATIM  \
0                replaced steering wheel now okay   
1                      steering wheel replacement   
2   replaced steering message no longer displayed   
3  remove and replace steering wheel wire harness   
4                         replaced steering wheel   

                                   CUSTOMER_VERBATIM REPAIR_DATE  \
0                        steering wheel coming apart  2024-01-02   
1  customer states the lettering and finish on th...  2024-01-04   
2  c/s: customer states the service driver assist...  2024-01-05   
3    horn and steering wheel switches are inoperable  2024-01-05   
4  special order part [ steering wheel heated ste...  2024-01-05   

                     CAUSAL_PAR

In [11]:
df.columns

Index(['VIN', 'TRANSACTION_ID', 'CORRECTION_VERBATIM', 'CUSTOMER_VERBATIM',
       'REPAIR_DATE', 'CAUSAL_PART_NM', 'GLOBAL_LABOR_CODE_DESCRIPTION',
       'PLATFORM', 'BODY_STYLE', 'VPPC', 'PLANT', 'BUILD_COUNTRY',
       'LAST_KNOWN_DLR_NAME', 'LAST_KNOWN_DLR_CITY', 'REPAIRING_DEALER_CODE',
       'DEALER_NAME', 'REPAIR_DLR_CITY', 'STATE', 'DEALER_REGION',
       'REPAIR_AGE', 'KM', 'COMPLAINT_CD_CSI', 'COMPLAINT_CD', 'VEH_TEST_GRP',
       'COUNTRY_SALE_ISO', 'ORD_SELLING_SRC_CD', 'OPTN_FAMLY_CERTIFICATION',
       'OPTF_FAMLY_EMISSIOF_SYSTEM', 'GLOBAL_LABOR_CODE',
       'TRANSACTION_CATEGORY', 'REPORTING_COST', 'TOTALCOST', 'LBRCOST',
       'ENGINE', 'ENGINE_DESC', 'TRANSMISSION', 'TRANSMISSION_DESC',
       'TRANSMISSION_TRACE_NBR', 'SRC_TXN_ID', 'SRC_VER_NBR',
       'TRANSACTION_CNTR', 'MEDIA_FLAG', 'VIN_MODL_DESGTR',
       'LAST_KNOWN_DELVRY_TYPE_CD', 'NON_CAUSAL_PART_QTY', 'SALES_REGION_CODE',
       'processed_text', 'tags'],
      dtype='object')

In [12]:
# Impute numerical columns with the median (for robust handling of outliers)
numerical_cols = ['REPORTING_COST', 'TOTALCOST', 'LBRCOST', 'KM', 'REPAIR_AGE', 'NON_CAUSAL_PART_QTY']
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())


In [13]:
# Impute categorical columns with the mode (most frequent value)
categorical_cols = ['CAUSAL_PART_NM', 'GLOBAL_LABOR_CODE_DESCRIPTION', 'PLATFORM', 'BODY_STYLE']
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


C:\Users\tanis\AppData\Local\Temp\ipykernel_7092\636782853.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


In [14]:
df['LAST_KNOWN_DLR_NAME'].fillna('UNKNOWN', inplace=True)
df['REPAIR_DLR_CITY'].fillna('UNKNOWN', inplace=True)


C:\Users\tanis\AppData\Local\Temp\ipykernel_7092\3836367367.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['LAST_KNOWN_DLR_NAME'].fillna('UNKNOWN', inplace=True)
C:\Users\tanis\AppData\Local\Temp\ipykernel_7092\3836367367.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [15]:
# Remove duplicate rows based on all columns
df.drop_duplicates(inplace=True)


In [16]:
# Convert date columns to datetime type
df['REPAIR_DATE'] = pd.to_datetime(df['REPAIR_DATE'], errors='coerce')

# Convert numeric columns to the appropriate type if they aren't already
df['TRANSACTION_ID'] = df['TRANSACTION_ID'].astype(int)
df['REPORTING_COST'] = df['REPORTING_COST'].astype(float)
df['TOTALCOST'] = df['TOTALCOST'].astype(float)


In [17]:
# Remove leading and trailing whitespaces in text columns
df['CUSTOMER_VERBATIM'] = df['CUSTOMER_VERBATIM'].str.strip()
df['CORRECTION_VERBATIM'] = df['CORRECTION_VERBATIM'].str.strip()

# Optional: Remove special characters if needed (for example, non-alphanumeric characters)
df['CUSTOMER_VERBATIM'] = df['CUSTOMER_VERBATIM'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df['CORRECTION_VERBATIM'] = df['CORRECTION_VERBATIM'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)


In [18]:
# Save the cleaned DataFrame to an Excel file
df.to_excel('cleaned_data_new.xlsx', index=False)
